In [1]:
from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display


In [13]:
## Test parameters



class Charger:

    def load_images(self):
        self.start = open("../images/green_on.png", "rb").read()
        self.reserve = open("../images/yellow_on.png", "rb").read()
        self.stop = open("../images/red_on.png", "rb").read()

    # Send a message to start charging
    def on_button_start(self, b):
        self.stm.send("start_charging")

    def on_button_reserve(self, b):
        self.stm.send("reserve_charger")

    def on_button_stop(self, b):
        self.stm.send("stop_charger")

    def display(self):
        # Buttons
        self.button_start = widgets.Button(description="Start charging")
        self.button_start.on_click(self.on_button_start)

        self.button_reserve = widgets.Button(description="Reserve charger")
        self.button_reserve.on_click(self.on_button_reserve)

        self.button_stop = widgets.Button(description="Stop charger")
        self.button_stop.on_click(self.on_button_stop)

        # Text field to display states
        self.text1 = widgets.Text(value='', placeholder='', description='State:', disabled = True)
        self.text2 = widgets.Text(value='', placeholder='', description='Enter reservation code:', disabled = False)
        display(self.text1, self.text2, self.button_start, self.button_reserve, self.button_stop)


    def __init__(self):
        self.load_images()
        self.display()
        global chargingcode_test
        chargingcode_test = "test"




    def func_available(self):
        self.text1.set_trait(name="value", value="Available")

    def func_reserved(self):
        self.text1.set_trait(name="value", value="Reserved")
        self.button_reserve.layout.visibility = 'hidden'

    def check_chargingcode(self):
        if self.text2.value == chargingcode_test:
            return 'charging'
        else:
            return 'reserved'

    def func_charging(self):
        self.text1.set_trait(name="value", value="Charging")

    def func_charging_battery_full(self):
        self.text1.set_trait(name="value", value="Charging with full battery")

    def func_overcharge(self):
        self.text1.set_trait(name="value", value="Overcharging")

    def func_finish_charging(self):
        self.text1.set_trait(name="value", value="Finished charging")

    def start_payment(self):
        return
    
    def stop_payment(self):
        return
    
    def start_overchargepayment(self):
        return
    
    def stop_overchargepayment(self):
        return
    
    def lock_plug(self):
        return
    
    def unlock_plug(self):
        return
    def charge_customer(self):
        return

    def start_power(self):
        return
    
    def stop_power(self):
        return

charger = Charger()
#Initial transition
t0 = {'source':'initial',
      'target':'available'}

#Transition from available to reserved
t1 = {'trigger':'reserve_charger',
      'source':'available',
      'target':'reserved'}


#Transition from availble to charging
t2 = {'trigger': 'start_charging',
      'source':'available',
      'target':'charging'}

#Transition from charging to finish_charging
t3 = {'trigger':'stop_charger',
      'source': 'charging',
      'target':'finish_charging'}

#Transition from charging to charging_battery_full
t4 = {'trigger':'t3',
      'source':'charging',
      'target':'charging_battery_full'}

#Transition from charging_battery_full to finish_charging
t5 = {'trigger':'stop_charger',
      'source':'charging_battery_full',
      'target':'finish_charging'}

#Transition from charging_battery_full to overcharge
t6 = {'trigger':'t2',
      'source':'charging_battery_full',
      'target':'overcharge'}

#Transition from overcharge to finish_charging
t7 = {'trigger':'stop_charger',
      'source':'overcharge',
      'target':'finish_charging'}

#Transition from finish_charging to available
t8 = {'trigger':'charger_available',
      'source':'finish_charging',
      'target':'available'}

#Transition from reserved to available
t9 = {'trigger':'t1',
      'source':'reserved',
      'target':'available'}

#Transition from reserved to charging
t10 = {'trigger':'start_charging',
       'source':'reserved',
       'function': charger.check_chargingcode,
       'effect':'stop_timer("t1")'}

# STATES

available = {'name':'available',
             'entry':'func_available' }

reserved = {'name':'reserved',
            'entry':'func_reserved; start_timer("t1", 10000)'}

charging = {'name':'charging',
            'entry':'func_charging; start_payment; lock_plug; start_power; start_timer("t3", 1000)'}

charging_battery_full = {'name':'charging_battery_full',
                         'entry':'func_charging_battery_full; start_timer("t2", 1000)'}

overcharge = {'name':'overcharge',
              'entry':'start_overchargepayment; func_overcharge'}

finish_charging = {'name':'finish_charging',
                   'entry':'func_finish_charging; stop_power; stop_payment; stop_overchargepayment; unlock_plug; charge_customer;'}



machine = Machine(name="charger", transitions=[t0,t1,t2,t3,t4,t5,t6,t7,t8,t9, t10], obj=charger, states=[available, reserved, charging, charging_battery_full, overcharge, finish_charging])
charger.stm = machine

driver = Driver()
driver.add_machine(machine)
driver.start()

Text(value='', description='State:', placeholder='')

Text(value='', description='Enter reservation code:', placeholder='')

Button(description='Start charging', style=ButtonStyle())

Button(description='Reserve charger', style=ButtonStyle())

Button(description='Stop charger', style=ButtonStyle())